**Algorithm 1**
Hamiltonian Monte Carlo

**function** Leapfrog $(\theta,r,\varepsilon)$

Set $\tilde{r} \leftarrow r + \frac{\varepsilon}{2} \nabla_\theta \mathcal{L}(\theta)$

Set $\tilde{\theta} \leftarrow \theta + \varepsilon \tilde{r}$

Set $\tilde{r} \leftarrow r + \frac{\varepsilon}{2} \nabla_\theta \mathcal{L}(\tilde{\theta})$

**return** $\tilde{\theta},\tilde{r}$

$\mathcal{L}$ is the logarithm of the joint density of the variables of interest $\theta$. The Leapfrog function of Algorithm 1 implements the Stormer-Verlet ("leapfrog") integrator, which proceeds according to the updates:

$r^{t + \frac{\varepsilon}{2}} = r^t + \frac{\varepsilon}{2} \nabla_\theta \mathcal{L}(\theta^t)$

$\theta^{t + \varepsilon} = \theta^t + \varepsilon r^{t + \frac{\varepsilon}{2}}$

$r^{t + \varepsilon} = r^{t + \frac{\varepsilon}{2}} + \frac{\varepsilon}{2} \nabla_\theta \mathcal{L}(\theta^{t + \varepsilon})$

where $r^t$ and $\theta^t$ denote the values of the momentum and position variables $r$ and $\theta$ at time $t$, $\nabla_\theta$ denotes the gradient with respect to $\theta$ and $\varepsilon$ is the step size parameter.

The performance of HMC depends strongly on choosing suitable values for $\varepsilon$ and $L$, the number of times chosen to run the leapfrog step. If $\varepsilon$ is too large, hen the simulation will be inaccurate and yield low acceptance rates. If $\varepsilon$ is too small, then computation will be wasted taking many small steps. If $L$ is too small, then successive samples will be close to one another, resulting in undesirable random walk behavior and slow mixing. If $L$ is too large, then HMC will generate trajectories that loop back and retrace